In [1]:
import torch as th 
from torch_geometric.data import Data, Batch
from torch_geometric.loader import DataLoader

from model import Generator, Discriminator, gradient_penalty
from utils import Transformed_PolyGraphDataset, CATEGORY_DICT

import numpy as np
import time 

c:\Users\haora\anaconda3\envs\graph_ml_course\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Hyperparameters


In [2]:
""" 
Hyperparameters
"""

MAX_POLYGONS = 30       # max nr. of polygons to output

# Optimizer params
g_lr = 0.001 
d_lr = 0.001
b1 = 0.5 
b2 = 0.999  

# WGAN params
N_critic = 5            # nr of times to train discriminator more
lambda_gp = 10          # gradient penalty hyperpraram

# Training params
MAX_EPOCHS = 50
BATCH_SIZE = 32

# Network parameters
NOISE_SIZE = 128
HIDDEN_GENERATOR = [64, 32, 32]                 # list of dimensions for hidden layers
OUTPUT_GENERATOR = MAX_POLYGONS * 2             # we want to output at most this many polygons per node (note [x1... y1...] format)

HIDDEN_DISCRIMINATOR = [64, 32, 16]

# Model definition

### Generator
- Input of Generator will always be: "noise size + Nr. of categories" 
- Output of Generator = (MAX_POYLGONS * 2) due to our output coordinate format


### Discriminator
- Input of Discriminator = output of Generator (MAX_POLYGONS * 2) due to our output coordinate format
- Output of Discriminator will always be 1

In [3]:
""" 
Model Definitions
"""
generator = Generator(input_dim=NOISE_SIZE + len(CATEGORY_DICT), 
                      output_dim=OUTPUT_GENERATOR, 
                      hidden_dims=HIDDEN_GENERATOR)

discriminator = Discriminator(input_dim=OUTPUT_GENERATOR, 
                              hidden_dims=HIDDEN_DISCRIMINATOR)

print(generator.module_list)
print(discriminator.module_list)

ModuleList(
  (0): TAGConv(141, 64, K=3)
  (1): TAGConv(64, 32, K=3)
  (2): TAGConv(32, 60, K=3)
)
ModuleList(
  (0): TAGConv(60, 64, K=3)
  (1): TAGConv(64, 32, K=3)
  (2): TAGConv(32, 16, K=3)
  (3): Linear(in_features=16, out_features=1, bias=True)
)


In [4]:
# Optimizers
optimizer_G = th.optim.Adam(generator.parameters(), lr=g_lr, betas=(b1, b2)) 
optimizer_D = th.optim.Adam(discriminator.parameters(), lr=d_lr, betas=(b1, b2))

# Training loop

Generator details:
- We generate a noise vector
- Generator uses noise vector and training data (the categories) as input
- Generator uses graph NN layers to generate coordinates out of the noise vectors using the graph structure and categories
- Generator output: (MAX_POLYGONS * 2) for each node in the input

Discriminator details:
- Generate discriminator output (score) for real data and fake (generated) data
- Compute loss over these scores with additional gradient penalty loss

In [12]:
def run_epoch(generator, discriminator, optimizer_g, optimizer_d, data_loader):

    losses_g, losses_d = [], []
    num_steps = 0

    # real = batch of Data() ex. [Data(), Data(), ...] is 1 batch
    start_time = time.time()
    for i, real in enumerate(data_loader):
        num_steps += 1

        # Input noise_data into generator            
        noise = th.randn((len(real.category), NOISE_SIZE))
        fake = generator(real, noise)     

        # fake.shape = (batch_size * nodes, output_features = 60)
        # We must turn this into appropriate (batch) input for the discriminator
        fake = Batch(geometry=fake, edge_index=real.edge_index, batch=real.batch)

        discriminator_fake = discriminator(fake)    # discriminator scores for fakes
        discriminator_real = discriminator(real)    # discriminator scores for reals
        
        gp = gradient_penalty(discriminator, real, fake)

        # Discriminator loss and train
        loss_discriminator = -(th.mean(discriminator_real) - th.mean(discriminator_fake)) + lambda_gp * gp
        
        discriminator.zero_grad() 
        loss_discriminator.backward() 
        optimizer_d.step()

        losses_d.append(loss_discriminator.item())

        # Only train Generator every 5 steps
        if num_steps % N_critic == 0:
            noise_g = th.randn((len(real.category), NOISE_SIZE))
            fake_g = generator(real, noise_g) 
            fake_g = Batch(geometry=fake_g, edge_index=real.edge_index, batch=real.batch)

            output = discriminator(fake_g).reshape(-1)      # discriminator scores for fake
            loss_generator = -th.mean(output)               # loss for genereator = the discriminators' judgement
                                                            # higher score = better
            generator.zero_grad()
            loss_generator.backward()
            optimizer_g.step()

            losses_g.append(loss_generator.item())
        
        # 49 is just a temporary number
        if i % 49 == 0:
            batch_time = time.time() - start_time
            print("Batch {} took {:.2f} seconds.".format(i, batch_time))
    
    return sum(losses_g) / len(losses_g), sum(losses_d) / len(losses_d)

# Main training loop

Code could be improved. Make sure to specify your own 'generator_path', 'discriminator_path'.

In [13]:
# Main training loop
def train(generator, discriminator, optimizer_g, optimizer_d, data_loader, max_epochs):
    start_time = time.time() 
    losses_g, losses_d = [], []

    for epoch in range(1, max_epochs + 1):
        epoch_time = time.time() 

        loss_g, loss_d = run_epoch(generator, discriminator, optimizer_g, optimizer_d, data_loader)
        losses_g.append(loss_g.item())
        losses_d.append(loss_d.item())

        # TODO: Evaluation and logging code??
        epoch_time = time.time() - epoch_time 
        total_time = time.time() - start_time
        print("Total runtime: {:.2f}, Epoch: {}, took: {:.2f} seconds, loss_g: {:.2f}, loss_d: {:.2f}".format(total_time, epoch, epoch_time, loss_g, loss_d))

        # save model every 10 epochs
        if epoch % 10:
            print("Saving model at epoch {}".format(epoch))
            generator_path = r'C:\School\DELFT\Graph_ML_project\saved_models\generator.pt'
            discriminator_path = r'C:\School\DELFT\Graph_ML_project\saved_models\discriminator.pt'
            th.save(generator.state_dict(), generator_path)
            th.save(discriminator.state_dict(), discriminator_path)

    return losses_g, losses_d

# Training code

In [14]:
path = r'C:\School\DELFT\Graph_ML_project\data\swiss-dwellings-v3.0.0'
dataset = Transformed_PolyGraphDataset(path)

dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)
n_batches = len(dataloader)
print(n_batches)

losses_g, losses_d = train(generator, discriminator, optimizer_G, optimizer_D, dataloader, max_epochs=20)

294
Batch 0 took 0.22 seconds.
Batch 49 took 9.63 seconds.
Batch 98 took 19.45 seconds.
Batch 147 took 29.03 seconds.
Batch 196 took 38.33 seconds.
Batch 245 took 47.74 seconds.
Total runtime: 56.96, Epoch: 1, took: 56.96 seconds, loss_g: -369.94, loss_d: -4111.66
Batch 0 took 0.21 seconds.
Batch 49 took 9.61 seconds.
Batch 98 took 19.33 seconds.
Batch 147 took 29.29 seconds.
Batch 196 took 39.20 seconds.
Batch 245 took 49.07 seconds.
Total runtime: 115.83, Epoch: 2, took: 58.86 seconds, loss_g: -248.54, loss_d: -3447.27
Batch 0 took 0.26 seconds.
Batch 49 took 10.38 seconds.
Batch 98 took 20.72 seconds.
Batch 147 took 31.20 seconds.
Batch 196 took 42.02 seconds.
Batch 245 took 53.28 seconds.
Total runtime: 179.80, Epoch: 3, took: 63.97 seconds, loss_g: -171.92, loss_d: -1748.79
Batch 0 took 0.27 seconds.
Batch 49 took 11.58 seconds.
Batch 98 took 23.00 seconds.
Batch 147 took 34.77 seconds.
Batch 196 took 46.50 seconds.
Batch 245 took 58.80 seconds.
Total runtime: 251.89, Epoch: 4, to

# Generator output code

The following section contains code to generate the coordinates for a floorplan using the generator.

requires:
- graph 
- categories per node (room)
- noise vector (of size: NOISE_SIZE)

In [5]:
# Load model
model_g_path = r'C:\School\DELFT\Graph_ML_project\saved_models\generator.pt'

# Make sure the model you're loading in is consistent with the size of your model
generator = Generator(input_dim=NOISE_SIZE + len(CATEGORY_DICT), 
                      output_dim=OUTPUT_GENERATOR, 
                      hidden_dims=HIDDEN_GENERATOR)

generator.load_state_dict(th.load(model_g_path))
print(generator)

Generator(
  (module_list): ModuleList(
    (0): TAGConv(141, 64, K=3)
    (1): TAGConv(64, 32, K=3)
    (2): TAGConv(32, 60, K=3)
  )
)


In [6]:
# Generate floorplan
path = r'C:\School\DELFT\Graph_ML_project\data\swiss-dwellings-v3.0.0'
dataset = Transformed_PolyGraphDataset(path)

data = dataset[0]
print(data)

noise_vector = th.randn((data.num_nodes, NOISE_SIZE))
output = generator(data, noise_vector)
print(output.shape)

output_data = Data(edge_index=data.edge_index, geometry=output, category=data.category, num_nodes=data.num_nodes)
print(output_data)

Data(edge_index=[2, 72], geometry=[42, 60], category=[42, 13], num_nodes=42)
torch.Size([42, 60])
Data(edge_index=[2, 72], geometry=[42, 60], category=[42, 13], num_nodes=42)


In [13]:
# Visualize floorplan
# use output_data = Data(edge_index, geometry, category, num_nodes)
from plot import plot_floorplan
import matplotlib.pyplot as plt


categories = output_data.category.argmax(dim=-1)
print(categories)
print(output_data.geometry)

tensor([3, 0, 1, 3, 1, 2, 3, 7, 4, 4, 3, 3, 4, 2, 0, 1, 3, 1, 0, 0, 2, 7, 4, 3,
        3, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5])
tensor([[-5.5891e+00,  2.9485e+00, -3.9081e-02,  ..., -1.7712e-01,
         -1.0998e-01, -2.2612e-02],
        [-3.7683e+00,  1.6112e+01, -2.5324e-03,  ..., -2.4961e-01,
         -4.7313e-02,  2.7180e-01],
        [ 8.0758e-01,  0.0000e+00, -0.0000e+00,  ..., -2.0717e-01,
         -1.2610e-01, -7.1626e-02],
        ...,
        [-7.5371e+00, -7.3328e+00, -7.2917e-02,  ..., -2.3652e-01,
         -0.0000e+00, -1.9689e-01],
        [-5.4364e+00,  3.5668e+00,  1.9171e-01,  ..., -0.0000e+00,
         -1.3893e-01, -1.0629e-01],
        [-0.0000e+00, -6.2440e+00, -1.0697e-01,  ..., -5.2443e-01,
         -1.3695e-01, -2.5121e-02]], grad_fn=<PreluKernelBackward0>)
